In [36]:
import pandas as pd
import pandas_ta as ta
import mplfinance as mpf
import numpy as np
import mplfinance as mpf
import matplotlib.pyplot as plt



In [45]:
AAPL_stock = pd.read_csv('/Users/hieudao/Desktop/ai/do_an/clean_data/clean_apple.csv')

AAPL_stock['Date'] = pd.to_datetime(AAPL_stock['Date'])
AAPL_stock = AAPL_stock.set_index('Date')

AAPL_stock.index

Index([1981-05-06 00:00:00-04:00, 1981-05-07 00:00:00-04:00,
       1981-05-08 00:00:00-04:00, 1981-05-11 00:00:00-04:00,
       1981-05-12 00:00:00-04:00, 1981-05-13 00:00:00-04:00,
       1981-05-14 00:00:00-04:00, 1981-05-15 00:00:00-04:00,
       1981-05-18 00:00:00-04:00, 1981-05-19 00:00:00-04:00,
       ...
       2023-06-23 00:00:00-04:00, 2023-06-26 00:00:00-04:00,
       2023-06-27 00:00:00-04:00, 2023-06-28 00:00:00-04:00,
       2023-06-29 00:00:00-04:00, 2023-06-30 00:00:00-04:00,
       2023-07-03 00:00:00-04:00, 2023-07-05 00:00:00-04:00,
       2023-07-06 00:00:00-04:00, 2023-07-07 00:00:00-04:00],
      dtype='object', name='Date', length=10626)

In [38]:
def calculate_ema(stock_data, N):
    if len(stock_data) < N:
        raise ValueError("Not enough data points to calculate EMA.")

    alpha = 2 / (N + 1)  # EMA smoothing factor
    ema_values = [stock_data['Close'].iloc[0]]

    for i in range(1, len(stock_data)):
        current_close = stock_data['Close'].iloc[i]
        ema = (current_close - ema_values[-1]) * alpha + ema_values[-1]
        ema_values.append(ema)

    return pd.Series(ema_values, index=stock_data.index)
ema_series = calculate_ema(AAPL_stock, 14)
AAPL_stock['ema'] = ema_series
AAPL_stock.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'NATR_3', 'RSI_3', 'ADX_3',
       'CCI_3_0.015', 'ROC_3', 'STOCHk_14_3_3', 'STOCHd_14_3_3', 'WILLR_3',
       'OBV', 'MACD_12_26_9', 'BBL_3_2.0', 'BBM_3_2.0', 'BBU_3_2.0',
       'BBB_3_2.0', 'BBP_3_2.0', 'min_price_3', 'max_price_3', 'mid_price',
       'tema_2', 'tema_100', 'tema_8', 'ema'],
      dtype='object')

In [39]:
mpf.plot(AAPL_stock[-100:],type='candle',
         #mav=(10),
         volume = True, 
         #show_nontrading=True,
        figsize=(30,20),
        addplot=[mpf.make_addplot(AAPL_stock['ema'])]
        )
       


TypeError: Expect data.index as DatetimeIndex

In [40]:
AAPL_stock.index

Index([1981-05-06 00:00:00-04:00, 1981-05-07 00:00:00-04:00,
       1981-05-08 00:00:00-04:00, 1981-05-11 00:00:00-04:00,
       1981-05-12 00:00:00-04:00, 1981-05-13 00:00:00-04:00,
       1981-05-14 00:00:00-04:00, 1981-05-15 00:00:00-04:00,
       1981-05-18 00:00:00-04:00, 1981-05-19 00:00:00-04:00,
       ...
       2023-06-23 00:00:00-04:00, 2023-06-26 00:00:00-04:00,
       2023-06-27 00:00:00-04:00, 2023-06-28 00:00:00-04:00,
       2023-06-29 00:00:00-04:00, 2023-06-30 00:00:00-04:00,
       2023-07-03 00:00:00-04:00, 2023-07-05 00:00:00-04:00,
       2023-07-06 00:00:00-04:00, 2023-07-07 00:00:00-04:00],
      dtype='object', name='Date', length=10626)